In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

from scipy import stats
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

np.random.seed(5)

In [52]:
# true parameters
mu = 75
sigma = 10

sigma_range = 5
sigma = widgets.IntSlider(value=sigma, min=sigma-sigma_range, max=sigma+sigma_range, description=r'$\sigma$')

# sample
sample_size = widgets.IntSlider(value=1000, min=100, max=2000, step=100, description=r'$n$')

sample = np.random.normal(mu, sigma.value, sample_size.value)
sample_mean = sample.mean()

# hypothesis
alpha = widgets.FloatSlider(value=0.05, min=0.01, max=0.1, step=0.01, description=r'$\alpha$')
mu_null = widgets.IntSlider(value=76, min=mu-5, max=mu+5, description=r'$\mu_0$')

# z-score
def z_score(sigma, sample_size, mu_null):
    global sample_mean
    return np.sqrt(sample_size) * (sample_mean - mu_null) / sigma

# power of z-test
def power(sigma, sample_size, mu_null, alpha):
    global mu
    shift = np.sqrt(sample_size) * (mu - mu_null) / sigma 
    quantile = stats.norm.ppf(1-alpha/2)
    return 1 - (stats.norm.cdf(quantile-shift) - stats.norm.cdf(-quantile-shift))

def print_score(sigma, sample_size, mu_null):
    print(f'z-score: {z_score(sigma, sample_size, mu_null)}')
    
def plot_score(sigma, sample_size, mu_null, alpha):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, title='Critical region')
    
    # bell curve
    x = np.linspace(-6, 6, 70) 
    ax.plot(x, stats.norm.pdf(x), color='blue')
    
    # z-score
    score = z_score(sigma, sample_size, mu_null)
    if -6 < score < 6:
        ax.plot(score, 0, 'o', color='cyan')
    
    # critical region edges
    quantile = -stats.norm.ppf(alpha/2)
    ax.axvline(-quantile, color='red', alpha=0.5)
    ax.axvline(quantile, color='red', alpha=0.5)
    
    # critical region areas 
    width = -quantile + 7
    left = Rectangle((-7, -1), width, 2)
    right = Rectangle((quantile, -1), width, 2)
    ax.add_collection(PatchCollection([left, right], color='red', alpha=0.08))
    
    return fig

def plot_power(sigma, sample_size, mu_null, alpha):
    global mu

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, title='Power analysis')
    
    scale = sigma / np.sqrt(sample_size)
    
    # true
    x = np.linspace(mu-6*scale, mu+6*scale, 70) 
    ax.plot(x, stats.norm.pdf(x, loc=mu, scale=scale), color='green')
    
    # hypo
    ax.plot(x, stats.norm.pdf(x, loc=mu_null, scale=scale), color='blue')
    
    # critical region
    scaled_ppf = stats.norm.ppf(alpha/2) * scale
    if mu-6*scale < mu_null-scaled_ppf < mu+6*scale:
        ax.axvline(mu_null-scaled_ppf, color='red', alpha=0.5)
    if mu-6*scale < mu_null+scaled_ppf < mu+6*scale:
        ax.axvline(mu_null+scaled_ppf, color='red', alpha=0.5)
    
    return fig

def print_probs(sigma, sample_size, mu_null, alpha):
    beta = np.round(1-power(sigma, sample_size, mu_null, alpha), 2)
    print(f'{"H null"} {True} {False}\n{"accept"} {1-alpha} {beta} \n{"reject"} {alpha} {1-beta} \n')

# outputs
output_score = widgets.interactive_output(plot_score, {'sigma': sigma, 'sample_size': sample_size, 'mu_null': mu_null, 'alpha': alpha})
output_power = widgets.interactive_output(plot_power, {'sigma': sigma, 'sample_size': sample_size, 'mu_null': mu_null, 'alpha': alpha})
output_probs = widgets.interactive_output(print_probs, {'sigma': sigma, 'sample_size': sample_size, 'mu_null': mu_null, 'alpha': alpha})

# layout
widgets.VBox([
    widgets.HBox([
        output_score, 
        output_power
    ]),
    widgets.HBox([
        widgets.VBox([sigma, sample_size, alpha, mu_null]), 
        widgets.VBox([widgets.Label('Conditional probabilty matrix'), output_probs])
    ])
])